In [3]:
import pandas as pd
from plant_model import PlantUnits
from utils import get_raw_data_by_time
from datetime import datetime, timedelta
import numpy as np
from utils import get_plant_characteristics
from utils import get_demand_data
from demand_model import Demand
from statistics import mean

In [4]:
COAL_RAMP_UP_PERCENT = 0.01
COAL_RAMP_DOWN_PERCENT = 0.015
COAL_EFFICIENCY_RATE = 0.55
DEVELOPMENT_PERIOD_START_TIME = datetime(2022, 1, 1)
DEVELOPMENT_PERIOD_END_TIME = datetime(2022, 2, 1)
MODEL_PERIOD_START_TIME = datetime(2021, 2, 1)
MODEL_PERIOD_END_TIME = datetime(2021, 3, 1)
INFINITE_CAPACITY = 8000
TIME_LEVEL = ['date','hour_of_day','time_block_of_day']
DEMAND_LEVEL = ['date','hour_of_day','avg_unit_current_load']
FILE_NAME = "RawData/upsldc_plant_unit_time_block.csv"


In [3]:
raw_data = pd.read_csv("RawData/upsldc_plant_unit_time_block.csv")
raw_data['date'] = pd.to_datetime(raw_data['data_capture_time_block_start']).dt.date

In [4]:
plant_unit_timeblocks = get_raw_data_by_time(raw_data,DEVELOPMENT_PERIOD_START_TIME, DEVELOPMENT_PERIOD_END_TIME)

In [5]:
model_demand = get_raw_data_by_time(raw_data,MODEL_PERIOD_START_TIME, MODEL_PERIOD_END_TIME)

In [6]:
plant_units = get_plant_characteristics(plant_unit_timeblocks)

/home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/ubuntu/UP-Power-Scheduling/utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_data['upsldc_unit_capacity'] = INFINITE_CAPACITY


In [7]:
demand_values = {} 
demand_of_UP_bydate_byhour_units,demand_UP = get_demand_data(model_demand)
for item in demand_of_UP_bydate_byhour_units:
    demand_key = str(item.date) + "-" +str(item.time_block)
    demand_values[demand_key] = item.demand_val

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 78 79 80 81]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [17]:
#finding average demand by hour for filling into the holes 
average_demand_list = []

for hour in range(0,24):
    for demand_block in demand_of_UP_bydate_byhour_units:
        if demand_block.hour == hour:
            average_demand_list.append(demand_block.demand_val)
    print(mean(average_demand_list))
    


9943.443192717174
9676.556619708976
9499.26258920351
9398.117516052907
9391.202775311198
9313.217321106304
9091.77588715781
8947.301237130228
9160.244041260305
9639.237359848168
10039.553339713571
10340.152811621401
10507.714729072499
10499.402018139319
10319.571521987617
10078.291474567728
9895.568904496793
9816.219716907346
9966.623067046756
10143.831875358484
10248.001507473089
10279.323040089215
10255.32140194397
10195.138820639173


In [8]:
#creating the LP problem with name "Production Cost Minimization Problem"
from pulp import *
prob = LpProblem("production_cost_minimization", LpMinimize)


In [9]:
#Reading data for the optimization problem

plant_names = []
plant_production_costs = {}
plant_capacity = {}
dates = []
for plant in plant_units:
    if(plant.average_variable_cost>0):

        plant_names.append(plant.name)
        plant_production_costs[plant.name] = plant.average_variable_cost
        plant_capacity[plant.name] = plant.capacity


scheduling_hours = list(range(0,24))
scheduling_dates =  demand_UP['date'].unique()


NameError: name 'demand_UP' is not defined

In [10]:
#Creating the production variables
production_vars = LpVariable.dicts(name = "production_units",indices = [(i,j,k) for i in plant_names for j in scheduling_dates for k in scheduling_hours],lowBound=0)

In [11]:
#LP Objective function 
prob += lpSum([plant_production_costs[i]*production_vars[(i,j,k)] for i in plant_names for j in scheduling_dates for k in scheduling_hours]), "Sum of production costs"

In [12]:
#Adding constraints to the model
#Demand constraints
#for every date, every hour, demand[date][hour] has to be satisfied
demand_values = {} 
for item in demand_of_UP_bydate_byhour_units:
 demand_key = str(item.date) + "-" +str(item.hour)
 demand_values[demand_key] = item.demand_val

for date in scheduling_dates:
   for hour in scheduling_hours:
      demand_date_hour = str(date) + "-"+str(hour)
      prob+= (lpSum(production_vars[(plant,date,hour)] for plant in plant_names) >= demand_values[demand_date_hour])


# capacity constraint 
# capacity of the power plant cannot be exceeded at any hour

for plant in plant_names:
   for date in scheduling_dates:
      for hour in scheduling_hours:
         prob += production_vars[(plant,date,hour)] <= plant_capacity[plant]






In [13]:
for plant in plant_units:
        if np.isnan(plant.capacity):
                print(plant.name)

In [14]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/b5da4c356935494581fb3b21240ec625-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b5da4c356935494581fb3b21240ec625-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46373 COLUMNS
At line 183462 RHS
At line 229831 BOUNDS
At line 229832 ENDATA
Problem MODEL has 46368 rows, 45696 columns and 91392 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 4.4607975 - largest nonzero change 2.9534216e-05 ( 0.0022632759%) - largest zero change 0
0  Obj 0 Primal inf 30203209 (672)
456  Obj 7986

-1

In [41]:
#printing important optimization values on screen 

#status of solving the optimization problem
print("Status:", LpStatus[prob.status])

#optimal objective value
print("Total cost of production = ", value(prob.objective))

output = ''
output += str(value(prob.objective)) + "\n"
for v in prob.variables():
    if v.varValue>0:
        output += v.name + "=" + str(v.varValue) + "\n" 



Status: Optimal
Total cost of production =  71460348.59357913


In [1]:
start_date = DEVELOPMENT_PERIOD_START_TIME
end_date = DEVELOPMENT_PERIOD_END_TIME
delta = timedelta(days=1)
while(end_date<start_date):
    day = start_date + delta
    print(day)



NameError: name 'DEVELOPMENT_PERIOD_START_TIME' is not defined

In [16]:
from datetime import date, timedelta
unique_time_blocks = []

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = MODEL_PERIOD_START_TIME
end_date = MODEL_PERIOD_END_TIME
for single_date in daterange(start_date, end_date):
    for time_block in range(1,97):
        unique_time_blocks.append(single_date.strftime("%Y-%m-%d") + str('-') + str(time_block))

['2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28']


In [14]:
model_demand

NameError: name 'model_demand' is not defined